In [1]:
DATASET_NAME = "CyberNative/Code_Vulnerability_Security_DPO"
MODEL_NAME = "Qwen/Qwen2.5-Coder-7B-Instruct"

# Get Code Snippets from Dataset

In [ ]:
# import librariesb
from datasets import load_dataset

/Users/ashle/anaconda3/envs/rl-sec/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
dataset = load_dataset(DATASET_NAME, split="train")

In [4]:
dataset

Dataset({
    features: ['lang', 'vulnerability', 'system', 'question', 'chosen', 'rejected'],
    num_rows: 4656
})

In [5]:
# each dataset is formatted as markdown with ```<language> in the beginning and ``` at the end
# we can extract the code snippets by removing these markers and save them to a new file inside a folder matching the language
# save c# as cs, fortran as f90, etc.
EXTENSIONS = {'c#': 'cs', 'fortran': 'f90', 'javascript': 'js', 'python': 'py', 'java': 'java', 'c++': 'cpp', 'c': 'c', 'ruby': 'rb', 'go': 'go', 'php': 'php', 'rust': 'rs', 'typescript': 'ts'}

OUTPUT_DIR="vulnerable_code_snippets/"
import os
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
for lang in set(dataset['lang']):
    lang_dir = os.path.join(OUTPUT_DIR, lang)
    if not os.path.exists(lang_dir):
        os.makedirs(lang_dir)
for idx, item in enumerate(dataset):
    code_snippet = item['rejected']
    lang = item['lang']
    # remove the ```<language> and ``` markers
    code_snippet = '\n'.join(code_snippet.split('\n')[1:-1])
    # save to file
    file_path = os.path.join(OUTPUT_DIR, lang, f"snippet_{idx}.{EXTENSIONS.get(lang, lang)}")
    with open(file_path, 'w') as f:
        f.write(code_snippet)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen2.5-Coder-7B-Instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use mps:0


In [ ]:
# prompt qwen with `question` field from dataset, save as json
dataset = load_dataset(DATASET_NAME, split="train")

pipe = pipeline("text-generation", model=MODEL_NAME)

import json
OUTPUT_PROMPT_FILE = "vulnerable_prompts.jsonl"
with open(OUTPUT_PROMPT_FILE, 'w') as f:
    for item in dataset:
        question = item['question']
        messages = [
            {"role": "user", "content": question},
        ]
        response = pipe(messages)
        json_line = {
            "question": question,
            "response": response
        }
        f.write(json.dumps(json_line) + '\n')
